In [1]:
from ff3 import FF3Cipher

In [22]:
import string

In [23]:
string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [2]:
key = "2DE79D232DF5585D68CE47882AE256D6"
tweak = "CBD09280979564"

In [51]:
c = FF3Cipher(key, tweak)

In [59]:
plaintext = "123456"
ciphertext = c.encrypt(plaintext)
decrypted = c.decrypt(ciphertext)

In [56]:
ciphertext

'609597486'

In [57]:
decrypted

'123456789'

In [99]:
c1 = FF3Cipher(key, tweak, radix=10)

In [100]:
c2 = FF3Cipher.withCustomAlphabet(key, tweak, string.ascii_letters)

In [102]:
plaintext = "AhmedZouari"
ciphertext = c6.encrypt(plaintext)
decrypted = c6.decrypt(ciphertext)

In [101]:
ciphertext

'DYrvtwnun'

In [80]:
decrypted

'AhmedZouari'

In [133]:
plain_text = '123 main st, apt 11'
plain_text = string.ascii_letters + string.digits + string.ascii_letters
plain_text = 'ahmed.s.zouari@gmail.com'

In [134]:
import re

In [135]:
pattern_nums = re.compile(r'\D+')

In [136]:
nums = pattern_nums.sub('', plain_text)
if len(nums) < 7:
    nums =  nums + (7-len(nums))*"7"
nums

'7777777'

In [137]:
pattern_letters = re.compile(r'[^a-zA-Z]')

In [138]:
letters = pattern_letters.sub('', plain_text)
letters

'ahmedszouarigmailcom'

In [139]:
ciphertext = c2.encrypt(letters)
ciphertext

'CjUwoljlbLXLVRVQbJhl'

In [132]:
ciphernum = c1.encrypt(nums)
ciphernum

'7207215089'

In [140]:
encrypted_text = []
letter_idx, digit_idx = 0, 0
for char in plain_text:
    if char in letters:
        encrypted_text.append(ciphertext[letter_idx])
        letter_idx += 1
    elif char in nums:
        encrypted_text.append(ciphernum[digit_idx])
        digit_idx += 1
    else:
        encrypted_text.append(char)
        
        

In [141]:
plain_text, ''.join(encrypted_text)

('ahmed.s.zouari@gmail.com', 'CjUwo.l.jlbLXL@VRVQb.Jhl')

In [124]:
string.ascii_letters + string.digits + string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [122]:
string.digits

'0123456789'